In [3]:
import pandas as pd
import os

def merge_datasets():
    """
    Une tres datasets CSV manteniendo solo las columnas comunes: subject, body, label
    """
    
    # Nombres de los archivos
    files = ['Nigerian_Fraud_es.csv', 'Ling_es.csv', 'CEAS_08_es.csv']
    
    # Columnas que queremos mantener
    target_columns = ['subject', 'body', 'label']
    
    # Lista para almacenar los dataframes procesados
    processed_datasets = []
    
    print("=== PROCESANDO ARCHIVOS ===")
    
    for file in files:
        try:
            # Verificar que el archivo existe
            if not os.path.exists(file):
                print(f"❌ Archivo no encontrado: {file}")
                continue
            
            # Leer el CSV
            print(f"\n📁 Procesando: {file}")
            df = pd.read_csv(file, encoding='utf-8')
            
            # Mostrar información del archivo original
            print(f"   Filas originales: {len(df)}")
            print(f"   Columnas originales: {list(df.columns)}")
            
            # Verificar que las columnas objetivo existen
            missing_columns = [col for col in target_columns if col not in df.columns]
            if missing_columns:
                print(f"   ⚠️  Columnas faltantes: {missing_columns}")
                
                # Intentar mapear nombres similares
                column_mapping = {}
                for col in missing_columns:
                    similar_cols = [c for c in df.columns if col.lower() in c.lower() or c.lower() in col.lower()]
                    if similar_cols:
                        print(f"   🔍 Columnas similares encontradas para '{col}': {similar_cols}")
                        # Aquí podrías agregar lógica para mapear automáticamente
                        # Por ahora, mostramos las opciones
                
                # Continuar con las columnas que sí existen
                available_columns = [col for col in target_columns if col in df.columns]
                if not available_columns:
                    print(f"   ❌ No se encontraron columnas objetivo en {file}")
                    continue
                
                df_filtered = df[available_columns].copy()
                
                # Agregar columnas faltantes con valores vacíos
                for col in missing_columns:
                    df_filtered[col] = ''
                    
            else:
                # Filtrar solo las columnas que necesitamos
                df_filtered = df[target_columns].copy()
            
            # Agregar columna de origen
            df_filtered['source_file'] = file.replace('.csv', '')
            
            # Limpiar datos
            df_filtered = df_filtered.dropna(subset=['subject', 'body'])  # Eliminar filas sin subject o body
            df_filtered = df_filtered.drop_duplicates()  # Eliminar duplicados
            
            # Limpiar espacios en blanco
            for col in ['subject', 'body', 'label']:
                if col in df_filtered.columns:
                    df_filtered[col] = df_filtered[col].astype(str).str.strip()
            
            processed_datasets.append(df_filtered)
            print(f"   ✅ Procesado: {len(df_filtered)} filas mantenidas")
            
        except Exception as e:
            print(f"   ❌ Error procesando {file}: {str(e)}")
            continue
    
    # Verificar que tenemos al menos un dataset procesado
    if not processed_datasets:
        print("\n❌ No se pudo procesar ningún archivo")
        return
    
    print(f"\n=== UNIENDO DATASETS ===")
    
    # Unir todos los datasets
    merged_df = pd.concat(processed_datasets, ignore_index=True)
    
    # Reordenar columnas
    column_order = ['subject', 'body', 'label', 'source_file']
    merged_df = merged_df[column_order]
    
    # Mostrar estadísticas finales
    print(f"📊 Dataset final:")
    print(f"   Total de filas: {len(merged_df)}")
    print(f"   Columnas: {list(merged_df.columns)}")
    
    # Mostrar distribución por archivo fuente
    print(f"\n📈 Distribución por archivo:")
    source_counts = merged_df['source_file'].value_counts()
    for source, count in source_counts.items():
        print(f"   {source}: {count} filas")
    
    # Mostrar distribución por etiqueta (si existe)
    if 'label' in merged_df.columns and not merged_df['label'].isna().all():
        print(f"\n🏷️  Distribución por etiqueta:")
        label_counts = merged_df['label'].value_counts()
        for label, count in label_counts.items():
            print(f"   {label}: {count} filas")
    
    # Guardar el archivo unificado
    output_file = 'dataset_unificado.csv'
    merged_df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"\n✅ Archivo guardado como: {output_file}")
    
    # Mostrar muestra de los datos
    print(f"\n📋 Muestra de los primeros 3 registros:")
    print(merged_df.head(3).to_string(index=False, max_colwidth=50))
    
    return merged_df

def main():
    """
    Función principal
    """
    print("🚀 INICIANDO PROCESO DE UNIÓN DE DATASETS")
    print("=" * 50)
    
    # Verificar que pandas está instalado
    try:
        import pandas as pd
        print("✅ Pandas disponible")
    except ImportError:
        print("❌ Error: pandas no está instalado")
        print("   Instala con: pip install pandas")
        return
    
    # Ejecutar el proceso
    result = merge_datasets()
    
    if result is not None:
        print("\n🎉 ¡Proceso completado exitosamente!")
        print(f"   El archivo 'dataset_unificado.csv' está listo para usar")
    else:
        print("\n❌ El proceso falló")

if __name__ == "__main__":
    main()

🚀 INICIANDO PROCESO DE UNIÓN DE DATASETS
✅ Pandas disponible
=== PROCESANDO ARCHIVOS ===

📁 Procesando: Nigerian_Fraud_es.csv
   Filas originales: 3332
   Columnas originales: ['sender', 'receiver', 'date', 'subject', 'body', 'urls', 'label']
   ✅ Procesado: 3194 filas mantenidas

📁 Procesando: Ling_es.csv
   Filas originales: 2859
   Columnas originales: ['subject', 'body', 'label']
   ✅ Procesado: 2790 filas mantenidas

📁 Procesando: CEAS_08_es.csv
   Filas originales: 39154
   Columnas originales: ['sender', 'receiver', 'date', 'subject', 'body', 'label', 'urls']
   ✅ Procesado: 35429 filas mantenidas

=== UNIENDO DATASETS ===
📊 Dataset final:
   Total de filas: 41413
   Columnas: ['subject', 'body', 'label', 'source_file']

📈 Distribución por archivo:
   CEAS_08_es: 35429 filas
   Nigerian_Fraud_es: 3194 filas
   Ling_es: 2790 filas

🏷️  Distribución por etiqueta:
   1: 21886 filas
   0: 19527 filas

✅ Archivo guardado como: dataset_unificado.csv

📋 Muestra de los primeros 3 regist